### Data 

The data for this project was downloaded from two sources. 

1) ATP Match Data
    - Includes player information and statistics for each match (aces, break points, etc)
    - https://github.com/JeffSackmann/tennis_atp 

2) Betting Odds Data
    - Also includes player information as well as prematch betting odds from various sources (sites differ depending on year)
    - http://www.tennis-data.co.uk/alldata.php

### Initial Data Cleaning
- Remove any exhibition/international matches (Davis Cup, Laver Cup, Next Gen Finals)

### Data Formatting

Since the data format is not consistent throughout, we have to clean this up as well. The end goal is to format enough columns properly to serve as keys when joining this dataframes together. Having enough keys ensures that each match is lined up correctly. 

In [1]:
import polars as pl
import math
from tennis_match_predictor.config import INTERIM_DATA_DIR

2025-08-18 23:01:56.134 | INFO     | tennis_match_predictor.config:<module>:11 - PROJ_ROOT path is: C:\Users\Admin\Projects\tennis-match-predictor


In [2]:
# Load ATP matches dataframe
df_atp_matches = pl.read_csv("../data/raw/atp_matches_raw.csv")

This dataset includes exhibition matches such as the Laver Cup and tournaments that do not count for ATP points i.e the Olympics. Since the betting odds dataset only includes official regulated ATP events, any non-ATP events will have to be dropped initially.

In [3]:
excluded_tournaments = [
    "Laver Cup",
    "Davis Cup", 
    "NextGen Finals",
    "Next Gen Finals",
    "Atp Cup",
    "United Cup",
    "Olympics"
]

# Use word boundaries (\b) to ensure exact matches and avoid partial matches
pattern = r"\b(" + "|".join(excluded_tournaments) + r")\b"

# Filter the dataframe of exhibition tournaments
df_atp_cleaned = df_atp_matches.filter(
    ~pl.col("tourney_name").str.contains(pattern, literal=False)
)

# Build list of tournament names
tourney_names_set = set(df_atp_cleaned["tourney_name"].to_list())
excluded_set = set(excluded_tournaments)

# Check that none of the excluded tournaments appear in the cleaned data
intersection = tourney_names_set & excluded_set
assert not intersection, f"Found excluded tournaments in cleaned data: {intersection}"

print(f"Original dataset: {len(df_atp_matches)} rows")
print(f"Cleaned dataset: {len(df_atp_cleaned)} rows") 
print(f"Removed: {len(df_atp_matches) - len(df_atp_cleaned)} rows")

Original dataset: 30573 rows
Cleaned dataset: 27530 rows
Removed: 3043 rows


In [4]:
# Load betting odds dataframe
df_betting_odds = pl.read_csv("../data/raw/betting_odds_raw.csv")

In [5]:
# Check if dataframes match length already
len(df_betting_odds) == len(df_atp_cleaned)

True

Since these dataframes are now the same size and presumably contain the same matches, we can now format some columns to serve as keys when joining the dataframes.

In [6]:
# See how the data is formatted in each column
df_atp_cleaned.head(1)

tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points,Year
str,str,str,i64,str,i64,i64,i64,i64,str,str,str,i64,str,f64,i64,i64,str,str,str,i64,str,f64,str,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""2014-339""","""Brisbane""","""Hard""",28,"""A""",20131229,1,103813,null,null,"""Jarkko Nieminen""","""L""",185,"""FIN""",32.4,105902,null,"""WC""","""James Duckworth""","""R""",183,"""AUS""",21.9,"""6-2 6-3""",3,"""R32""",72,5,3,71,54,36,9,9,5,5,9,7,51,29,23,6,8,2,5,39,1090,136,425,2014


In [7]:
df_betting_odds.head(1)

ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,Loser,WRank,LRank,WPts,LPts,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment,B365W,B365L,EXW,EXL,LBW,LBL,PSW,PSL,SJW,SJL,MaxW,MaxL,AvgW,AvgL,Year
i64,str,str,str,str,str,str,str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str,str,str,i64,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
1,"""Brisbane""","""Brisbane International""","""2013-12-30""","""ATP250""","""Outdoor""","""Hard""","""1st Round""",3,"""Matosevic M.""","""Benneteau J.""",61,35,774,1160,5,7,6,4,7,6,null,null,null,null,2,1,"""Completed""",1.72,2.0,1.8,2.0,1.91,1.8,1.88,2.01,1.91,1.8,2.0,2.1,1.82,1.94,2014


Despite containing the same information, all the columns are formatted differently. Even the tournament name is different as Jeff Sackmann uses the location/venue as the name while the betting odds dataset often uses the official name (with sponsors).

To start, the winner/loser name will be standardized across both datasets. Since the betting odds dataframe only uses initials while the ATP matches uses the full name, we will format the ATP matches dataset to be of the form last name, first name initial. This will ensure that it is an exact match, as some players may have the same last name. 

In [8]:
def format_player_name(name: str) -> str:
    """
    Convert full name into 'LastName F.' format.
    
    Examples:
    - Roger Federer -> Federer R.
    - Alex De Minaur -> De Minaur A.
    """
    parts = name.strip().split()
    if len(parts) == 1:
        return name

    first, last = parts[0], parts[1:]
    return " ".join(last) + f" {first[0]}."
    
df_atp_cleaned = df_atp_cleaned.with_columns([
    pl.col("winner_name").map_elements(format_player_name, return_dtype=pl.String).alias("w_name_key"), 
    pl.col("loser_name").map_elements(format_player_name, return_dtype=pl.String).alias("l_name_key")
])

For the best accuracy, we have to make sure that the dataframes are joined at the correct match. Since both datasets vary in how tournaments are named (and would take a while to manually rename), the next best thing is determining the round. Although players can play each other several times a year, it is often at different stages of a tournament (finals, semifinals, etc). Formatting round names for both datasets will ensure better matches.

In [9]:
def normalize_round_name(round_str: str, draw_size: int) -> str:
    """Convert round acronyms (R128, R64, etc.) into full round names, using draw size."""
    
    fixed_round_names = {
        "F": "The Final",
        "SF": "Semifinals",
        "QF": "Quarterfinals",
        "RR": "Round Robin",
    }
    if round_str in fixed_round_names:
        return fixed_round_names[round_str]
    
    # Handle numbered rounds (R128, R64, R32, etc.)
    if round_str.startswith("R"):
        try:
            remaining_num_players = int(round_str[1:])  # e.g. "R32" -> 32
        except ValueError:
            return round_str  
        
        # Each round eliminates half the players -> log2
        total_rounds = math.ceil(math.log2(draw_size))
        round_number = total_rounds - int(math.log2(remaining_num_players)) + 1
        
        suffix = {1 : "st", 2 : "nd", 3 : "rd"}.get(round_number, "th")  # 4th, 5th...
        return f"{round_number}{suffix} Round"
    
    return round_str 

df_atp_cleaned = df_atp_cleaned.with_columns(
    pl.struct(["round", "draw_size"])
    .map_elements(
        lambda x: normalize_round_name(x["round"], x["draw_size"]), 
        return_dtype=pl.String
    )
    .alias("round_key")
)

In [10]:
df_combined = df_atp_cleaned.join(df_betting_odds, 
                    how="left", 
                    left_on=["Year", "best_of", "w_name_key", "l_name_key", "surface", "round_key", "winner_rank", "loser_rank", "winner_rank_points", "loser_rank_points"],
                    right_on=["Year", "Best of", "Winner", "Loser", "Surface", "Round", "WRank", "LRank", "WPts", "LPts"],
                    )

assert len(df_combined) == len(df_atp_cleaned) == len(df_betting_odds)

We now have successfully joined both datasets to include all match statistics and betting odds. This will be the dataset used for further analysis.

In [11]:
# Add dataset to data/interim for future use
output_path = INTERIM_DATA_DIR / "joined_dataset_raw.csv"
df_combined.write_csv(output_path)